In [229]:
import os
import csv
import json
import datetime
import requests
import geopandas
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [210]:
# New Column Names
new_date_clmn_nm = "date"
new_st_clmn_nm = "state"
new_pop_at_hm_clmn_nm = "pop_at_home"
new_pop_not_at_hm_clmn_nm = "pop_not_at_home"
new_nmbr_trps_clmn_nm = "trips"
new_cases_clmn_nm = "covid_cases"
new_dths_clmn_nm = "covid_deaths"

In [ ]:
# File Name for the Final COVID DataFrame
# dtfrm_file_nm = "ASN_Table.csv"

# ========================================

# Importing Data

# ========================================

# ---------------------------------------------------------------------

## Import bts.gov Trip Data

# ---------------------------------------------------------------------

In [2]:
# File Path for Importing the bts.gov Trips Data .csv File
bts_gov_trps_file_pth = os.path.join("Resources", "bts.gov", "Trips_by_Distance.csv")

# Convert the CSV to a Dataframe
bts_gov_trps_df = pd.read_csv(bts_gov_trps_file_pth)

In [ ]:
bts_gov_trps_df.head()

In [ ]:
print(bts_gov_trps_df.count())

In [3]:
bts_gov_trps_df.dtypes

Level                              object
Date                               object
State FIPS                        float64
State Postal Code                  object
County FIPS                       float64
County Name                        object
Population Staying at Home         object
Population Not Staying at Home     object
Number of Trips                    object
Number of Trips <1                 object
Number of Trips 1-3                object
Number of Trips 3-5                object
Number of Trips 5-10               object
Number of Trips 10-25              object
Number of Trips 25-50              object
Number of Trips 50-100             object
Number of Trips 100-250            object
Number of Trips 250-500            object
Number of Trips >=500              object
dtype: object

# ---------------------------------------------------------------------

## Import New York Times COVID-19 Data

# ---------------------------------------------------------------------

In [175]:
# File Path for Importing the New York Times COVID-19 Data .csv File
nytimes_covid_19_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20200121_to_20201130.csv")

# Convert the CSV to a Dataframe
nytimes_covid_19_df = pd.read_csv(nytimes_covid_19_file_pth)

In [113]:
nytimes_covid_19_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
784208,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6.0
784209,2020-11-30,Teton,Wyoming,56039.0,1644,2.0
784210,2020-11-30,Uinta,Wyoming,56041.0,1070,4.0
784211,2020-11-30,Washakie,Wyoming,56043.0,440,8.0


In [114]:
nytimes_covid_19_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [174]:
nytimes_covid_19_df.keys()

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [183]:
nytimes_covid_19_df_test = nytimes_covid_19_df.rename(columns = {"date": "date_covid", "state": "state_covid", "cases": "cases_covid", "deaths": "deaths_covid"})
nytimes_covid_19_df_test.keys()
# nytimes_covid_19_df_test = nytimes_covid_19_df_test["date_covid", "state_covid", "cases_covid","deaths_covid"]

Index(['date_covid', 'county', 'state_covid', 'fips', 'cases_covid',
       'deaths_covid'],
      dtype='object')

# ---------------------------------------------------------------------

# ******NOT WORKING******

# ---------------------------------------------------------------------

In [200]:
# How to save only selected columns of a DataFrame to HDF5
# https://stackoverflow.com/questions/27878780/pandas-how-to-save-only-selected-columns-of-a-dataframe-to-hdf5
cols_to_keep = [['date', 'state', 'cases', 'deaths']]
nytimes_covid_19_df.loc[:, cols_to_keep].to_hdf(...)

KeyError: "None of [Index([('date', 'state', 'cases', 'deaths')], dtype='object')] are in the [columns]"

In [209]:
nytimes_covid_19_df = nytimes_covid_19_df[['date', 'state', 'cases', 'deaths']]
# nytimes_covid_19_df = nytimes_covid_19_df["date", "state", "cases", "deaths"]
nytimes_covid_19_df

,date,state,cases,deaths
0,2020-01-21,Washington,1,0.0
1,2020-01-22,Washington,1,0.0
2,2020-01-23,Washington,1,0.0
3,2020-01-24,Illinois,1,0.0
4,2020-01-24,Washington,1,0.0
...,...,...,...,...
784208,2020-11-30,Wyoming,1888,6.0
784209,2020-11-30,Wyoming,1644,2.0
784210,2020-11-30,Wyoming,1070,4.0
784211,2020-11-30,Wyoming,440,8.0


# ---------------------------------------------------------------------

In [115]:
# Shape of the New York Times COVID-19 Dataframe
# len(nytimes_covid_19_df)
nytimes_covid_19_df.shape

(784213, 6)

In [116]:
# Number of Rows that do Contain Null Values
print(nytimes_covid_19_df.count())

date      784213
county    784213
state     784213
fips      776761
cases     784213
deaths    767870
dtype: int64


In [117]:
# Data Types in the New York Dataframe
nytimes_covid_19_df.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths    float64
dtype: object

# ---------------------------------------------------------------------

# Import US State Name and Postal Code Data
# From the U.S. Bureau of Labor Statistics (bls.gov)

# ---------------------------------------------------------------------

In [201]:
bls_gov_st_st_code_df = pd.read_html("https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes")
len(bls_gov_st_st_code_df)

2

In [202]:
bls_gov_st_st_code_df = bls_gov_st_st_code_df[0]
bls_gov_st_st_code_df

,State/District,Abbreviation,Postal Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


# ---------------------------------------------------------------------

# Import County Polygons .json File

# ---------------------------------------------------------------------

In [231]:
# File Path for Importing the County Geometry Polygons.json File
cnty_geo_file_pth = os.path.join("Resources", "County_Geometry", "county_geometry.json")

# File Path for Importing the Example Househole Income Geometry Polygons.json File
household_income_geo_file_pth = os.path.join("Resources", "County_Geometry", "Median_Household_Income_2016.geojson")

# Convert the CSV to a Dataframe
household_income_covid_19_df = geopandas.read_file(household_income_geo_file_pth)
household_income_covid_19_df

,OBJECTID,GEOID10,COUNTYFP10,TRACTCE10,Cnty_Name,City_Name,ZIP,Acres,SqMi,Pop_16,...,UnEmp_Rate,Emp_15,Emp_Den,Emp_Src,Income_Pct,Income_Rpt,Income_Src,Under19_Pc,MHI2016,geometry
0,11,06025010101,025,010101,Imperial,,92233,93903.349290,146.723983,4223,...,0.101266,1454,9.909764,LEHD 2015,0,,,2.0,NaN,"POLYGON ((-115.64096 33.35902, -115.64096 33.3..."
1,213,06059980000,059,980000,,Anaheim,92802,683.188267,1.067482,25,...,0.000000,30340,28422.033762,LEHD 2015,0,,,60.0,NaN,"POLYGON ((-117.91511 33.81539, -117.91537 33.8..."
2,229,06059021813,059,021813,,Anaheim,92807,1159.678407,1.811998,102,...,0.000000,19437,10726.835928,LEHD 2015,0,,,30.4,NaN,"POLYGON ((-117.83125 33.86212, -117.83125 33.8..."
3,833,06071980100,071,980100,,San Bernardino,92408,1982.388981,3.097483,0,...,0.000000,11200,3615.839308,LEHD 2015,0,,,0.0,NaN,"POLYGON ((-117.25971 34.10356, -117.25971 34.1..."
4,878,06071012300,071,012300,,San Bernardino,92346,211.696199,0.330775,4400,...,0.269231,2952,8924.487123,LEHD 2015,0,,,0.0,NaN,"POLYGON ((-117.22623 34.13691, -117.22623 34.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3946,3947,06037430302,037,430302,Los Angeles,Monrovia,91016,478.859944,0.748219,5841,...,0.069227,706,943.574433,LEHD 2015,100238,2011-2015,ACS_15_5YR_DP03,20.2,65976.0,"POLYGON ((-118.00434 34.15133, -118.00413 34.1..."
3947,3948,06037430723,037,430723,Los Angeles,Arcadia,91007,120.125030,0.187695,4663,...,0.100369,527,2807.741229,LEHD 2015,65625,2011-2015,ACS_15_5YR_DP03,27.2,73068.0,"POLYGON ((-118.05228 34.12915, -118.05185 34.1..."
3948,3949,06037431100,037,431100,Los Angeles,Monrovia,91016,744.710275,1.163610,7157,...,0.085203,9186,7894.398928,LEHD 2015,54613,2011-2015,ACS_15_5YR_DP03,27.8,67750.0,"POLYGON ((-117.99500 34.13045, -117.99446 34.1..."
3949,3950,06037533603,037,533603,Los Angeles,Bell,90201,199.808992,0.312202,7114,...,0.131519,2807,8990.986769,LEHD 2015,41368,2011-2015,ACS_15_5YR_DP03,33.8,41325.0,"POLYGON ((-118.19135 33.96894, -118.19144 33.9..."


# ========================================

# Null Values

# ========================================

# ---------------------------------------------------------------------

## bts.gov Trip Data Null Values

# ---------------------------------------------------------------------

In [8]:
# Returns true if the column has null values
bts_gov_trps_df.isnull().any()
# OR
# bts_gov_trps_df.isnull().any(axis=0) #(axis=0 is the defult)

Level                             False
Date                              False
State FIPS                         True
State Postal Code                  True
County FIPS                        True
County Name                        True
Population Staying at Home         True
Population Not Staying at Home     True
Number of Trips                    True
Number of Trips <1                 True
Number of Trips 1-3                True
Number of Trips 3-5                True
Number of Trips 5-10               True
Number of Trips 10-25              True
Number of Trips 25-50              True
Number of Trips 50-100             True
Number of Trips 100-250            True
Number of Trips 250-500            True
Number of Trips >=500              True
dtype: bool

In [54]:
# Rows with Null Values in the Trips Dataframe 
bts_gov_trps_df.isnull().sum()

Level                                 0
Date                                  0
State FIPS                          691
State Postal Code                   691
County FIPS                       35932
County Name                       35932
Population Staying at Home        23111
Population Not Staying at Home    23111
Number of Trips                   23111
Number of Trips <1                23111
Number of Trips 1-3               23111
Number of Trips 3-5               23111
Number of Trips 5-10              23111
Number of Trips 10-25             23111
Number of Trips 25-50             23111
Number of Trips 50-100            23111
Number of Trips 100-250           23111
Number of Trips 250-500           23111
Number of Trips >=500             23111
dtype: int64

In [9]:
# Returns a Dataframe Containing Only the Rows With Null Values
bts_gov_trps_df_null_val = bts_gov_trps_df[bts_gov_trps_df.isnull().any(axis=1)]

# Returns the Number of Cells Within a Column With Null Values
print(bts_gov_trps_df_null_val.count())
bts_gov_trps_df_null_val

Level                             59043
Date                              59043
State FIPS                        58352
State Postal Code                 58352
County FIPS                       23111
County Name                       23111
Population Staying at Home        35932
Population Not Staying at Home    35932
Number of Trips                   35932
Number of Trips <1                35932
Number of Trips 1-3               35932
Number of Trips 3-5               35932
Number of Trips 5-10              35932
Number of Trips 10-25             35932
Number of Trips 25-50             35932
Number of Trips 50-100            35932
Number of Trips 100-250           35932
Number of Trips 250-500           35932
Number of Trips >=500             35932
dtype: int64


,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
1,County,2019/01/01,2.0,AK,2164.0,Lake and Peninsula Borough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,County,2019/01/01,2.0,AK,2230.0,Skagway Municipality,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,County,2019/01/01,2.0,AK,2282.0,Yakutat City and Borough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,County,2019/01/01,15.0,HI,15005.0,Kalawao County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
824,County,2019/01/01,31.0,NE,31005.0,Arthur County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207049,National,2020/11/17,NaN,NaN,NaN,NaN,"97,573,501","229,593,933","989,568,973","234,805,101","259,934,985","122,714,018","150,290,148","145,112,889","49,573,515","16,971,134","7,726,729","1,711,083","729,371"
2207050,National,2020/11/18,NaN,NaN,NaN,NaN,"95,954,135","231,213,299","1,027,444,626","248,373,480","269,543,706","126,367,337","154,455,563","149,041,180","51,172,462","17,999,411","7,953,887","1,787,348","750,252"
2207051,National,2020/11/19,NaN,NaN,NaN,NaN,"93,468,789","233,698,645","1,023,244,380","247,800,254","266,987,840","125,041,236","152,756,826","147,639,120","52,272,385","20,187,095","8,050,777","1,765,481","743,366"
2207052,National,2020/11/20,NaN,NaN,NaN,NaN,"92,068,777","235,098,657","956,927,423","223,417,886","244,773,240","117,098,300","143,579,157","139,779,548","53,139,601","23,609,611","8,890,314","1,864,123","775,643"


# ---------------------------------------------------------------------

### Dateframe with Totals for each State and the Nation Per Day

# ---------------------------------------------------------------------

In [10]:
# Array of the Columns Where the Null Values will be Converted to 0
# These are the Columns that Should Have a Zero Values Instead of a Null Values
clmns_to_cnvrt_null_vls_to_0 =["Population Staying at Home", "Population Not Staying at Home","Number of Trips","Number of Trips <1", "Number of Trips 1-3", "Number of Trips 1-3", "Number of Trips 3-5", "Number of Trips 5-10", "Number of Trips 10-25", "Number of Trips 25-50", "Number of Trips 50-100", "Number of Trips 100-250", "Number of Trips 250-500", "Number of Trips >=500"]

In [11]:
# Drop the Rows that With Null Values (NaN) in ALL the Columns in the ["clmns_to_cnvrt_null_vls_to_0"] array: if All the Values are Null.
bts_gov_trps_df_null_val_lvl_st_ntnl = bts_gov_trps_df_null_val.dropna(how="all", subset=clmns_to_cnvrt_null_vls_to_0)
print(bts_gov_trps_df_null_val_lvl_st_ntnl.count())
bts_gov_trps_df_null_val_lvl_st_ntnl

Level                             35932
Date                              35932
State FIPS                        35241
State Postal Code                 35241
County FIPS                           0
County Name                           0
Population Staying at Home        35932
Population Not Staying at Home    35932
Number of Trips                   35932
Number of Trips <1                35932
Number of Trips 1-3               35932
Number of Trips 3-5               35932
Number of Trips 5-10              35932
Number of Trips 10-25             35932
Number of Trips 25-50             35932
Number of Trips 50-100            35932
Number of Trips 100-250           35932
Number of Trips 250-500           35932
Number of Trips >=500             35932
dtype: int64


,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
15710,State,2019/01/01,25.0,MA,NaN,NaN,"1,798,521","5,082,532","22,181,890","6,078,316","5,980,407","2,680,470","3,106,759","2,817,590","1,021,091","316,642","104,691","18,791","57,133"
15711,State,2019/01/01,26.0,MI,NaN,NaN,"2,342,813","7,622,550","26,701,254","6,633,681","6,893,858","3,358,485","4,024,493","3,681,856","1,264,454","479,687","242,283","55,384","67,073"
15712,State,2019/01/01,27.0,MN,NaN,NaN,"1,266,769","4,327,262","15,278,584","3,589,616","3,806,602","1,772,986","2,246,949","2,419,710","830,948","337,906","187,197","42,524","44,146"
15713,State,2019/01/01,28.0,MS,NaN,NaN,"596,999","2,380,405","7,297,125","1,787,312","1,930,713","884,893","1,102,272","934,487","355,845","178,107","92,480","24,396","6,620"
15714,State,2019/01/01,29.0,MO,NaN,NaN,"1,530,765","4,576,960","16,618,431","4,033,851","4,385,048","2,036,854","2,526,033","2,313,883","809,030","290,444","154,169","39,949","29,170"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207049,National,2020/11/17,NaN,NaN,NaN,NaN,"97,573,501","229,593,933","989,568,973","234,805,101","259,934,985","122,714,018","150,290,148","145,112,889","49,573,515","16,971,134","7,726,729","1,711,083","729,371"
2207050,National,2020/11/18,NaN,NaN,NaN,NaN,"95,954,135","231,213,299","1,027,444,626","248,373,480","269,543,706","126,367,337","154,455,563","149,041,180","51,172,462","17,999,411","7,953,887","1,787,348","750,252"
2207051,National,2020/11/19,NaN,NaN,NaN,NaN,"93,468,789","233,698,645","1,023,244,380","247,800,254","266,987,840","125,041,236","152,756,826","147,639,120","52,272,385","20,187,095","8,050,777","1,765,481","743,366"
2207052,National,2020/11/20,NaN,NaN,NaN,NaN,"92,068,777","235,098,657","956,927,423","223,417,886","244,773,240","117,098,300","143,579,157","139,779,548","53,139,601","23,609,611","8,890,314","1,864,123","775,643"


#### The Above Dataframe Looks Like the Totals for Each State and the Nation Per Day.

# ---------------------------------------------------------------------

### Dataframe With Each State's Total Trips Per Day

# ---------------------------------------------------------------------

In [14]:
# Dataframe With the Total State Trips Per Day
bts_gov_trps_df_cnty_clmn_null_lvl_st = bts_gov_trps_df_null_val_lvl_st_ntnl.dropna(how="all", subset=["State Postal Code"])
print(bts_gov_trps_df_cnty_clmn_null_lvl_st.count())
bts_gov_trps_df_cnty_clmn_null_lvl_st

Level                             35241
Date                              35241
State FIPS                        35241
State Postal Code                 35241
County FIPS                           0
County Name                           0
Population Staying at Home        35241
Population Not Staying at Home    35241
Number of Trips                   35241
Number of Trips <1                35241
Number of Trips 1-3               35241
Number of Trips 3-5               35241
Number of Trips 5-10              35241
Number of Trips 10-25             35241
Number of Trips 25-50             35241
Number of Trips 50-100            35241
Number of Trips 100-250           35241
Number of Trips 250-500           35241
Number of Trips >=500             35241
dtype: int64


,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
15710,State,2019/01/01,25.0,MA,NaN,NaN,"1,798,521","5,082,532","22,181,890","6,078,316","5,980,407","2,680,470","3,106,759","2,817,590","1,021,091","316,642","104,691","18,791","57,133"
15711,State,2019/01/01,26.0,MI,NaN,NaN,"2,342,813","7,622,550","26,701,254","6,633,681","6,893,858","3,358,485","4,024,493","3,681,856","1,264,454","479,687","242,283","55,384","67,073"
15712,State,2019/01/01,27.0,MN,NaN,NaN,"1,266,769","4,327,262","15,278,584","3,589,616","3,806,602","1,772,986","2,246,949","2,419,710","830,948","337,906","187,197","42,524","44,146"
15713,State,2019/01/01,28.0,MS,NaN,NaN,"596,999","2,380,405","7,297,125","1,787,312","1,930,713","884,893","1,102,272","934,487","355,845","178,107","92,480","24,396","6,620"
15714,State,2019/01/01,29.0,MO,NaN,NaN,"1,530,765","4,576,960","16,618,431","4,033,851","4,385,048","2,036,854","2,526,033","2,313,883","809,030","290,444","154,169","39,949","29,170"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207042,State,2020/11/21,10.0,DE,NaN,NaN,"277,495","689,676","2,832,707","604,972","713,195","369,273","454,354","401,164","180,043","91,437","16,663",915,691
2207043,State,2020/11/21,23.0,ME,NaN,NaN,"405,767","932,637","3,482,073","636,056","851,371","440,434","555,144","603,887","243,648","110,970","33,916","4,585","2,062"
2207044,State,2020/11/21,40.0,OK,NaN,NaN,"1,121,125","2,821,954","10,725,736","2,421,533","2,736,594","1,229,055","1,588,849","1,638,856","616,443","346,034","122,738","20,359","5,275"
2207045,State,2020/11/21,55.0,WI,NaN,NaN,"1,708,357","4,105,211","15,988,642","3,415,712","3,963,118","1,993,093","2,440,392","2,442,194","990,260","508,393","184,244","41,181","10,055"


In [15]:
# Which Columns Have Null Values in the State Level Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st.isnull().any()

Level                             False
Date                              False
State FIPS                        False
State Postal Code                 False
County FIPS                        True
County Name                        True
Population Staying at Home        False
Population Not Staying at Home    False
Number of Trips                   False
Number of Trips <1                False
Number of Trips 1-3               False
Number of Trips 3-5               False
Number of Trips 5-10              False
Number of Trips 10-25             False
Number of Trips 25-50             False
Number of Trips 50-100            False
Number of Trips 100-250           False
Number of Trips 250-500           False
Number of Trips >=500             False
dtype: bool

#### Correct, Only the County Columns have Null Values. Since This is a Daily State Totals Dataframe. 

# ---------------------------------------------------------------------

### Dataframe With Total National Trips Per Day

# ---------------------------------------------------------------------

In [ ]:
# Dataframe the Total National Trips Per Day
bts_gov_trps_df_NULL_VALUES_lvl_ntnl = bts_gov_trps_df_NULL_VALUES_lvl_st_ntnl[bts_gov_trps_df_NULL_VALUES_lvl_st_ntnl["Level"] == "National"]
print(bts_gov_trps_df_NULL_VALUES_lvl_ntnl.count())
bts_gov_trps_df_NULL_VALUES_lvl_ntnl

In [ ]:
# Which Columns Have Null Values in the National Level Dataframe
bts_gov_trps_df_NULL_VALUES_lvl_ntnl.isnull().any()

#### Correct, the State and County Columns have Null Values. Since This is a Daily National Totals Dataframe. 

In [ ]:
bts_gov_trps_df_NULL_VALUES_lvl_ntnl = bts_gov_trps_df_NULL_VALUES_lvl_ntnl

# ---------------------------------------------------------------------

### Dataframe With Total County Trips Per Day

# ---------------------------------------------------------------------

In [ ]:
# Dataframe the Total County Trips Per Day
bts_gov_trps_df_lvl_cnty = bts_gov_trps_df[bts_gov_trps_df["Level"] == "County"]
print(bts_gov_trps_df_lvl_cnty.count())
bts_gov_trps_df_lvl_cnty

In [ ]:
# Which Columns Have Null Values in the County Level Dataframe
bts_gov_trps_df_lvl_cnty.isnull().any()

#### #### I am Not Sure Why the Population and Trips Columns Have Null Values. I am Assuming They are Null Because No Trips Were Taken?

In [ ]:
bts_gov_trps_df_NULL_VALUES_null_dltd["Level"].unique()

# ---------------------------------------------------------------------

#### Convert All the Null Values in the "clmns_to_cnvrt_null_vls_to_0" Array to Zero

# ---------------------------------------------------------------------

In [ ]:
bts_gov_trps_df_lvl_cnty[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_lvl_cnty[clmns_to_cnvrt_null_vls_to_0].replace(np.nan, 0)
bts_gov_trps_df_lvl_cnty

In [ ]:
# Which Columns Have Null Values in the County Level Dataframe
bts_gov_trps_df_lvl_cnty.isnull().any()

#### Correct, Now all the Columns Have a Value.

# ---------------------------------------------------------------------

## New York Times COVID-19 Data Null Values

# ---------------------------------------------------------------------

In [118]:
# Columns with Null Values in the New York Times Dataframe 
nytimes_covid_19_df.isnull().any()

date      False
county    False
state     False
fips       True
cases     False
deaths     True
dtype: bool

In [119]:
# Rows with Null Values in the New York Times Dataframe 
nytimes_covid_19_df.isnull().sum()

date          0
county        0
state         0
fips       7452
cases         0
deaths    16343
dtype: int64

In [120]:
# Returns a Dataframe Containing Only the Rows With Null Values
nytimes_covid_19_df_null_val = nytimes_covid_19_df[nytimes_covid_19_df.isnull().any(axis=1)]

# Returns the Number of Cells Within a Column With Null Values
print(nytimes_covid_19_df_null_val.count())
nytimes_covid_19_df_null_val

date      23795
county    23795
state     23795
fips      16343
cases     23795
deaths     7452
dtype: int64


,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,NaN,1,0.0
418,2020-03-01,Unknown,Rhode Island,NaN,2,0.0
448,2020-03-02,New York City,New York,NaN,1,0.0
450,2020-03-02,Unknown,Rhode Island,NaN,2,0.0
482,2020-03-03,New York City,New York,NaN,2,0.0
...,...,...,...,...,...,...
783378,2020-11-30,Unknown,Rhode Island,NaN,10879,83.0
783579,2020-11-30,Unknown,Tennessee,NaN,1108,3.0
783866,2020-11-30,Unknown,Utah,NaN,185,2.0
783883,2020-11-30,Unknown,Vermont,NaN,8,0.0


#### Since the ["fips"] Column has a Lot of Null Values, Lets Isolate the ["deaths"] Column to See How many Null Values are  in the ["deaths"] Column.

In [121]:
# Isolated the Null Values in the ["deaths"] Column.
nytimes_covid_19_df_dth_clmn_null_val = nytimes_covid_19_df[nytimes_covid_19_df["deaths"].isnull()]
print(len(nytimes_covid_19_df_dth_clmn_null_val))
print(nytimes_covid_19_df_dth_clmn_null_val["state"].value_counts())
nytimes_covid_19_df_dth_clmn_null_val

16343
Puerto Rico    16343
Name: state, dtype: int64


,date,county,state,fips,cases,deaths
117486,2020-05-05,Adjuntas,Puerto Rico,72001.0,3,NaN
117487,2020-05-05,Aguada,Puerto Rico,72003.0,7,NaN
117488,2020-05-05,Aguadilla,Puerto Rico,72005.0,11,NaN
117489,2020-05-05,Aguas Buenas,Puerto Rico,72007.0,22,NaN
117490,2020-05-05,Aibonito,Puerto Rico,72009.0,13,NaN
...,...,...,...,...,...,...
783369,2020-11-30,Vega Baja,Puerto Rico,72145.0,1702,NaN
783370,2020-11-30,Vieques,Puerto Rico,72147.0,110,NaN
783371,2020-11-30,Villalba,Puerto Rico,72149.0,483,NaN
783372,2020-11-30,Yabucoa,Puerto Rico,72151.0,585,NaN


# *******Puerto Rico has no reported deaths?*******

# ---------------------------------------------------------------------

#### Convert All the Null Values in the ["deaths"] Column to Zero

# ---------------------------------------------------------------------

In [122]:
# Replace the Null Values With 0
nytimes_covid_19_df_no_null_val_dths = nytimes_covid_19_df
nytimes_covid_19_df_no_null_val_dths["deaths"] = nytimes_covid_19_df_no_null_val_dths["deaths"].replace(np.nan, 0)
nytimes_covid_19_df_no_null_val_dths

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
784208,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6.0
784209,2020-11-30,Teton,Wyoming,56039.0,1644,2.0
784210,2020-11-30,Uinta,Wyoming,56041.0,1070,4.0
784211,2020-11-30,Washakie,Wyoming,56043.0,440,8.0


In [123]:
# Rows with Null Values
nytimes_covid_19_df_no_null_val_dths[nytimes_covid_19_df_no_null_val_dths["deaths"].isnull()]

,date,county,state,fips,cases,deaths


# ========================================

# Convert the Date Column to a datetime Date Type & Convert the Other Columns with Numbers into an int64 Data Type

# ========================================

# ---------------------------------------------------------------------

## bts.gov Trip Data

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

### The State Level Dataframe

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

#### Remove the Commas in the [clmns_to_cnvrt_null_vls_to_0] Columns Array in the State Level Dataframe

# ---------------------------------------------------------------------

In [16]:
# Remove the Commas from the [clmns_to_cnvrt_null_vls_to_0] Array.
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms = bts_gov_trps_df_cnty_clmn_null_lvl_st
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms[clmns_to_cnvrt_null_vls_to_0].replace(',','', regex=True)
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms

/Users/vincentadams/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
15710,State,2019/01/01,25.0,MA,NaN,NaN,1798521,5082532,22181890,6078316,5980407,2680470,3106759,2817590,1021091,316642,104691,18791,57133
15711,State,2019/01/01,26.0,MI,NaN,NaN,2342813,7622550,26701254,6633681,6893858,3358485,4024493,3681856,1264454,479687,242283,55384,67073
15712,State,2019/01/01,27.0,MN,NaN,NaN,1266769,4327262,15278584,3589616,3806602,1772986,2246949,2419710,830948,337906,187197,42524,44146
15713,State,2019/01/01,28.0,MS,NaN,NaN,596999,2380405,7297125,1787312,1930713,884893,1102272,934487,355845,178107,92480,24396,6620
15714,State,2019/01/01,29.0,MO,NaN,NaN,1530765,4576960,16618431,4033851,4385048,2036854,2526033,2313883,809030,290444,154169,39949,29170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207042,State,2020/11/21,10.0,DE,NaN,NaN,277495,689676,2832707,604972,713195,369273,454354,401164,180043,91437,16663,915,691
2207043,State,2020/11/21,23.0,ME,NaN,NaN,405767,932637,3482073,636056,851371,440434,555144,603887,243648,110970,33916,4585,2062
2207044,State,2020/11/21,40.0,OK,NaN,NaN,1121125,2821954,10725736,2421533,2736594,1229055,1588849,1638856,616443,346034,122738,20359,5275
2207045,State,2020/11/21,55.0,WI,NaN,NaN,1708357,4105211,15988642,3415712,3963118,1993093,2440392,2442194,990260,508393,184244,41181,10055


In [18]:
# View Data Types
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms.dtypes

Level                              object
Date                               object
State FIPS                        float64
State Postal Code                  object
County FIPS                       float64
County Name                        object
Population Staying at Home         object
Population Not Staying at Home     object
Number of Trips                    object
Number of Trips <1                 object
Number of Trips 1-3                object
Number of Trips 3-5                object
Number of Trips 5-10               object
Number of Trips 10-25              object
Number of Trips 25-50              object
Number of Trips 50-100             object
Number of Trips 100-250            object
Number of Trips 250-500            object
Number of Trips >=500              object
dtype: object

In [19]:
# Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64 = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64[clmns_to_cnvrt_null_vls_to_0].astype("int64")

/Users/vincentadams/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# ---------------------------------------------------------------------

#### Convert the ["Date"] Column to a datetime64[ns] Data Type in the State Level Dataframe

# ---------------------------------------------------------------------

In [27]:
# Convert the ["Date"] Column from a String Data Type to a datetime64[ns] Data Type
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"]  = pd.to_datetime(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"])

print(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"].dtypes)
print("-------------------------")
print(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"])

datetime64[ns]
-------------------------
15710     2019-01-01
15711     2019-01-01
15712     2019-01-01
15713     2019-01-01
15714     2019-01-01
             ...    
2207042   2020-11-21
2207043   2020-11-21
2207044   2020-11-21
2207045   2020-11-21
2207046   2020-11-21
Name: Date, Length: 35241, dtype: datetime64[ns]


/Users/vincentadams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm.dtypes

Level                                     object
Date                              datetime64[ns]
State FIPS                               float64
State Postal Code                         object
County FIPS                              float64
County Name                               object
Population Staying at Home                 int64
Population Not Staying at Home             int64
Number of Trips                            int64
Number of Trips <1                         int64
Number of Trips 1-3                        int64
Number of Trips 3-5                        int64
Number of Trips 5-10                       int64
Number of Trips 10-25                      int64
Number of Trips 25-50                      int64
Number of Trips 50-100                     int64
Number of Trips 100-250                    int64
Number of Trips 250-500                    int64
Number of Trips >=500                      int64
dtype: object

# ---------------------------------------------------------------------

## New York Times COVID-19 Data

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

### Change Deaths From a float to an int Data Type in the COVID-19 Dataframe

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

#### Check the Values After the Decimal to Determine if They are Greater Than 0.

# ---------------------------------------------------------------------

In [128]:
# Check the Numbers After a Decimal Point
# https://stackoverflow.com/questions/57900886/how-to-check-numbers-after-decimal-point

nytimes_covid_19_df_no_null_val_dths_no_dcml_dths = nytimes_covid_19_df_no_null_val_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths['check_point'] = np.modf(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths['deaths'])[0].ne(0).astype(int)
# nytimes_covid_19_df_no_null_val_dths_dcml_chck = nytimes_covid_19_df_no_null_val_dths_dcml_chck['check_point']
nytimes_covid_19_df_no_null_val_dths_dcml_chck_dcml_not_zero = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths[nytimes_covid_19_df_no_null_val_dths_no_dcml_dths["check_point"]!=0]
nytimes_covid_19_df_no_null_val_dths_dcml_chck_dcml_not_zero
# nytimes_covid_19_df_no_null_val_dths_dcml_chck["state"].value_counts()

,date,county,state,fips,cases,deaths,check_point


# Before Rounding, Need to Convert the ["deaths] Column Datatype From a String to a int

In [136]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths.dtypes

date            object
county          object
state           object
fips           float64
cases            int64
deaths         float64
check_point      int64
dtype: object

In [138]:
# Convert to ["deaths"] Column to an int Data Type
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths["deaths"] = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths["deaths"].astype("int64")
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths.dtypes

date            object
county          object
state           object
fips           float64
cases            int64
deaths           int64
check_point      int64
dtype: object

In [144]:
# Round the Death Rate Up to a Whole Number.
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths
# nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_rnd_ceil["deaths"] = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_rnd_ceil["deaths"].apply(np.ceil)

nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths.round({"deaths": 0})
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths

,date,county,state,fips,cases,deaths,check_point
0,2020-01-21,Snohomish,Washington,53061.0,1,0,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0,0
3,2020-01-24,Cook,Illinois,17031.0,1,0,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0,0
...,...,...,...,...,...,...,...
784208,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6,0
784209,2020-11-30,Teton,Wyoming,56039.0,1644,2,0
784210,2020-11-30,Uinta,Wyoming,56041.0,1070,4,0
784211,2020-11-30,Washakie,Wyoming,56043.0,440,8,0


# ---------------------------------------------------------------------

### Convert the ["Date"] Column to a datetime64[ns] Data Type in the State Level Dataframe

# ---------------------------------------------------------------------

In [146]:
# Convert the ["Date"] Column from a String Data Type to a datetime64[ns] Data Type
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"]  = pd.to_datetime(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"])

print(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"].dtypes)
print("-------------------------")
print(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"])

datetime64[ns]
-------------------------
0        2020-01-21
1        2020-01-22
2        2020-01-23
3        2020-01-24
4        2020-01-24
            ...    
784208   2020-11-30
784209   2020-11-30
784210   2020-11-30
784211   2020-11-30
784212   2020-11-30
Name: date, Length: 784213, dtype: datetime64[ns]


In [147]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date.dtypes

date           datetime64[ns]
county                 object
state                  object
fips                  float64
cases                   int64
deaths                  int64
check_point             int64
dtype: object

In [151]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date

,date,county,state,fips,cases,deaths,check_point
0,2020-01-21,Snohomish,Washington,53061.0,1,0,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0,0
3,2020-01-24,Cook,Illinois,17031.0,1,0,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0,0
...,...,...,...,...,...,...,...
784208,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6,0
784209,2020-11-30,Teton,Wyoming,56039.0,1644,2,0
784210,2020-11-30,Uinta,Wyoming,56041.0,1070,4,0
784211,2020-11-30,Washakie,Wyoming,56043.0,440,8,0


# ========================================

# Dataframes with Limited Columns

# ========================================

# ---------------------------------------------------------------------

## State Dataframe with Limited Columns

# ---------------------------------------------------------------------

In [33]:
# New State Dataframe With the Following Columns: ["Date", "State Postal Code", "Population Staying at Home", "Population Not Staying at Home","Number of Trips"] 
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn[["Date", "State Postal Code", "Population Staying at Home", "Population Not Staying at Home","Number of Trips"]]
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn

,Date,State Postal Code,Population Staying at Home,Population Not Staying at Home,Number of Trips
15710,2019-01-01,MA,1798521,5082532,22181890
15711,2019-01-01,MI,2342813,7622550,26701254
15712,2019-01-01,MN,1266769,4327262,15278584
15713,2019-01-01,MS,596999,2380405,7297125
15714,2019-01-01,MO,1530765,4576960,16618431
...,...,...,...,...,...
2207042,2020-11-21,DE,277495,689676,2832707
2207043,2020-11-21,ME,405767,932637,3482073
2207044,2020-11-21,OK,1121125,2821954,10725736
2207045,2020-11-21,WI,1708357,4105211,15988642


In [34]:
# Check the Data Types of the Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.dtypes

Date                              datetime64[ns]
State Postal Code                         object
Population Staying at Home                 int64
Population Not Staying at Home             int64
Number of Trips                            int64
dtype: object

In [35]:
# Check the Dataframe for Null Values
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.isnull().any()

Date                              False
State Postal Code                 False
Population Staying at Home        False
Population Not Staying at Home    False
Number of Trips                   False
dtype: bool

In [41]:
# Number of Unique Values in the Date Column of the State Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn["Date"].value_counts()

2020-10-28    51
2020-09-08    51
2019-06-10    51
2020-01-11    51
2019-06-23    51
              ..
2020-04-23    51
2019-12-05    51
2020-08-23    51
2020-07-07    51
2019-02-04    51
Name: Date, Length: 691, dtype: int64

In [43]:
# Number of Non-NA cells for each column in the Date Column of the State Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.count()

Date                              35241
State Postal Code                 35241
Population Staying at Home        35241
Population Not Staying at Home    35241
Number of Trips                   35241
dtype: int64

In [44]:
# Total Number of Rows in the Dataframe
len(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn)

35241

In [45]:
# The Shape of the Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.shape

(35241, 5)

# ---------------------------------------------------------------------

## COVID-19 Dataframe with Limited Columns

# ********NOT WORKING********

# ---------------------------------------------------------------------

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date.

In [161]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date_lmtd_clmn = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date", "cases"]
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date_lmtd_clmn

KeyError: ('date', 'cases')

# ========================================

# Change the Name of the Columns

# ========================================

# ---------------------------------------------------------------------

## Change the Column Names in the State Trips Dataframe

# ---------------------------------------------------------------------

In [214]:
# New Column Names
new_date_clmn_nm = "date"
new_st_clmn_nm = "state"
new_st_pstl_code_clmn_nm = "state_postal_code"
new_pop_at_hm_clmn_nm = "pop_at_home"
new_pop_not_at_hm_clmn_nm = "pop_not_at_home"
new_nmbr_trps_clmn_nm = "number_trips"
new_cases_clmn_nm = "covid_cases"
new_dths_clmn_nm = "covid_deaths"

In [212]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.keys()

Index(['Date', 'State Postal Code', 'Population Staying at Home',
       'Population Not Staying at Home', 'Number of Trips'],
      dtype='object')

In [227]:
# Changed the Names of the Column Headers
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng ({"Date": new_date_clmn_nm, "State Postal Code": new_st_pstl_code_clmn_nm, "Population Staying at Home": new_pop_at_hm_clmn_nm, "Population Not Staying at Home": new_pop_not_at_hm_clmn_nm, "Number of Trips": new_nmbr_trps_clmn_nm}, axis = 1)
# bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng#.head()

TypeError: 'DataFrame' object is not callable

# ========================================

# Add the State Name & State Postal Code

# ========================================

# ---------------------------------------------------------------------

## Add the State Postal Code in the Trips Dataframe

# ---------------------------------------------------------------------

In [204]:

bls_gov_st_st_code_df.keys()

Index(['State/District', 'Abbreviation', 'Postal Code'], dtype='object')

In [205]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.keys()

Index(['Date', 'State Postal Code', 'Population Staying at Home',
       'Population Not Staying at Home', 'Number of Trips'],
      dtype='object')

In [ ]:
# Replace all the Abbreviations with the Full State Name 
for t in range(len(State_Abbreviations_Code_Table_df)):

    for i in range(len(avg_auto_insnc_concat_df_clnd)):

        if State_Abbreviations_Code_Table_df.iloc[t]["Code"] == avg_auto_insnc_concat_df_clnd.iloc[i]["State"]:
            avg_auto_insnc_concat_df_clnd.iloc[i]["State"] = State_Abbreviations_Code_Table_df.iloc[t]["Description"]

In [ ]:
# ********NOT WORKING********

# ========================================

# Change the Name of the Dataframes

# ========================================

# ---------------------------------------------------------------------

## Change the State Trips Dataframe

# ---------------------------------------------------------------------

In [211]:
bts_gov_trps_df_FINAL = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn
bts_gov_trps_df_FINAL

,Date,State Postal Code,Population Staying at Home,Population Not Staying at Home,Number of Trips
15710,2019-01-01,MA,1798521,5082532,22181890
15711,2019-01-01,MI,2342813,7622550,26701254
15712,2019-01-01,MN,1266769,4327262,15278584
15713,2019-01-01,MS,596999,2380405,7297125
15714,2019-01-01,MO,1530765,4576960,16618431
...,...,...,...,...,...
2207042,2020-11-21,DE,277495,689676,2832707
2207043,2020-11-21,ME,405767,932637,3482073
2207044,2020-11-21,OK,1121125,2821954,10725736
2207045,2020-11-21,WI,1708357,4105211,15988642


# ========================================

In [10]:
# Array of the Columns Where the Null Values will be Converted to 0
# These are the Columns that Should Have a Zero Values Instead of a Null Values
clmns_to_cnvrt_null_vls_to_0 =["Population Staying at Home", "Population Not Staying at Home","Number of Trips","Number of Trips <1", "Number of Trips 1-3", "Number of Trips 1-3", "Number of Trips 3-5", "Number of Trips 5-10", "Number of Trips 10-25", "Number of Trips 25-50", "Number of Trips 50-100", "Number of Trips 100-250", "Number of Trips 250-500", "Number of Trips >=500"]

In [ ]:
# Count the Null Columns
null_columns = bts_gov_trps_df.columns[bts_gov_trps_df.isnull().any()]
bts_gov_trps_df[null_columns].isnull().sum()

In [ ]:
# Array of the Columns Where the Null Values will be Converted to 0
clmns_to_dlt_st_ntnl_totls =["County FIPS", "County Name"]

In [ ]:
# Drop the Rows that Contains Null (NaN) Values in the ["County FIPS"] and ["County Name"] Columns.
bts_gov_trps_df_clnd = bts_gov_trps_df.dropna(how="all", subset=clmns_to_dlt_st_ntnl_totls)
print(bts_gov_trps_df_clnd.count())
bts_gov_trps_df_clnd

In [ ]:
bts_gov_trps_df_clnd.dtypes

In [ ]:
bts_gov_trps_df_clnd_test.dtypes

In [ ]:
bts_gov_trps_df_clnd_test_grpby = bts_gov_trps_df_clnd_test.groupby(["State Postal Code", "County Name"])
bts_gov_trps_df_clnd_test_grpby

In [ ]:
# Create Some Plots to Look at the Data

date_fltr = (bts_gov_trps_df_clnd_test['Date'] <'2020-01-01') | (bts_gov_trps_df_clnd_test['Date']>'2020-11-21')
    
#     plt.scatter(x = group[date_filter_not]['air_pressure(mBar)'],
#                 y = group[date_filter_not]["wind_speed"],

for title, group in bts_gov_trps_df_clnd_test.groupby(["County FIPS"]):
    plt.plot(group[date_fltr]['Date'], 
             group[date_fltr]['Number of Trips']) 
#              subplots=True, 
#              title=title)
    fig = plt.gcf()
    
    plt.figure(figsize=(200,100))
#     fig.savefig(f"../images/{title}_wind_speed.png")

In [ ]:
# Filter Data on ["State Postal Code"] Column for "AK"
ak_df = bts_gov_trps_df_clnd_test[bts_gov_trps_df_clnd_test["County Name"] == "Adair County"]
ak_df

In [ ]:
st_cnt = bts_gov_trps_df["Population Staying at Home"].unique()
print(st_cnt)

In [ ]:
bts_gov_trps_df["Population Staying at Home"].value_counts()

In [ ]:
bts_gov_trps_df["VA Trip Totals"] = bts_gov_trps_df["Number of Trips <1"] + bts_gov_trps_df["Number of Trips 1-3"] + bts_gov_trps_df["Number of Trips 3-5"] +  bts_gov_trps_df["Number of Trips 5-10"] + bts_gov_trps_df["Number of Trips 10-25"] + bts_gov_trps_df["Number of Trips 25-50"] + bts_gov_trps_df["Number of Trips 50-100"] +bts_gov_trps_df["Number of Trips 100-250"] +bts_gov_trps_df["Number of Trips 250-500"] +bts_gov_trps_df["Number of Trips >=500"]

bts_gov_trps_df.head()

In [ ]:
# List of all the Columns in the Trips Dataframe
bts_gov_trps_df.columns

In [ ]:
bts_gov_trps_df.describe()

In [ ]:
bts_gov_trps_df["Number of Trips"].describe()

In [ ]:
# File Path for Importing the January 21, 2020 thru November 30, 2020 COVID-19 Data .csv File
NY_Tmes_COVID_20200121_to_20201130_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20200121_to_20201130.csv")

# Convert the CSV to a Dataframe
NY_Tmes_COVID_20200121_to_20201130_df = pd.read_csv(NY_Tmes_COVID_20200121_to_20201130_file_pth)

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.head()

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.tail()

In [ ]:
# # File Path for Importing the January 21, 2020 thru November 30, 2020 COVID-19 Data .csv File
# NY_Tmes_COVID_20201201_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20201201.csv")

# # Convert the CSV to a Dataframe
# NY_Tmes_COVID_20201201__df = pd.read_csv(NY_Tmes_COVID_20201201_file_pth)

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.head()

In [ ]:
# # API Keys for the Zip-Codes API
# key = "?key="
# user_API_key = "QSTIRXQR7CN0Q8T9UGRT"
# url = "https://api.zip-codes.com/ZipCodesAPI.svc/1.0/QuickGetZipCodeDetails/"

# print(url + key + user_API_key)

In [ ]:
# New Column Header Names
zip_clmn_hdr_nm = "Zip_Code"
avg_auto_insrnc_rate_clmn_hdr_nm = "Average_Auto_Insurance_Rate"

In [ ]:
# The Number of Tables on Each Website
print(len(ASN_Table_2020))
# print(len(AutoInsurance_Tables))
# print(len(State_Abbreviations_Code_Table))

# Tables from Car Insurance (www.carinsurance.com)

## The Most and Lease Expensive Auto Insurance

In [ ]:
# Table of The Most and Lease Expensive Auto Insurance, Table #5, from CarInsurance (www.carinsurance.com)
mst_and_lst_expnsv_by_state_CarInsurance_df = CarInsurance_Tables[5]
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

# Resaved the DataFrame With Only the Columns Needed 
mst_and_lst_expnsv_by_state_CarInsurance_df = mst_and_lst_expnsv_by_state_CarInsurance_df[["State", "ZIP Code", "City", "Average Annual Rate"]]
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

# Changed the Names of the Column Headers
mst_and_lst_expnsv_by_state_CarInsurance_df = mst_and_lst_expnsv_by_state_CarInsurance_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Rate" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

## The Most Expensive Auto Insurance

In [ ]:
# Table of The Most Expensive Auto Insurance, Table #6, from CarInsurance (www.carinsurance.com)
mst_expnsv_by_state_CarInsurance_df = CarInsurance_Tables[6]
mst_expnsv_by_state_CarInsurance_df.head()

# Resaved the DataFrame With Only the Columns Needed 
mst_expnsv_by_state_CarInsurance_df = mst_expnsv_by_state_CarInsurance_df[["State", "ZIP Code", "City", "Average Annual Rate"]]
mst_expnsv_by_state_CarInsurance_df.head()

# Changed the Names of the Column Headers
mst_expnsv_by_state_CarInsurance_df = mst_expnsv_by_state_CarInsurance_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Rate" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
mst_expnsv_by_state_CarInsurance_df.head()

# Tables from Auto Insurance (www.autoinsurance.org)

## The 25 Most Expensive Auto Insurance

In [ ]:
# Table of The 25 Most Expensive Auto Insurance, Table #3, from AutoInsurance (www.autoinsurance.org)
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = AutoInsurance_Tables[3]
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Changed the Names of the Column Headers
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Add a New Column Called State, the Column is Empty
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df["State"] = ""
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Reorder the Columns
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df[["State", zip_clmn_hdr_nm, "City", avg_auto_insrnc_rate_clmn_hdr_nm]]
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

## The 25 Lease Expensive Auto Insurance

In [ ]:
# Table of The 25 Lowest Expensive Auto Insurance, Table #4, from AutoInsurance (www.autoinsurance.org)
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = AutoInsurance_Tables[4]
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Changed the Names of the Column Headers
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Add a New Column Called State, the Column is Empty
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df["State"] = ""
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Reorder the Columns
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df[["State", zip_clmn_hdr_nm, "City", avg_auto_insrnc_rate_clmn_hdr_nm]]
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

## The Highest and Lowest Auto Insurance Rates

In [ ]:
# Table of The Highest and Lowest Auto Insurance Rate, Table #5, from AutoInsurance (www.autoinsurance.org)
hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df = AutoInsurance_Tables[5]
hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

### Create a DataFrame for the Least Expensive Auto Insurance Rates from hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df (Above)

In [ ]:
# Splitting Up The Table of The Highest and Lowest Auto Insurance Rate
lwst_avg_annl_auto_insrnc_rates_per_state_df = hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df[["State", "Least Expensive Zip Code", "City", "Lowest Average Annual Auto Insurance Rates"]]
lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Reorganize Column Headers and Rename Some of the Column Headers
lwst_avg_annl_auto_insrnc_rates_per_state_df = lwst_avg_annl_auto_insrnc_rates_per_state_df.rename ({"Least Expensive Zip Code" : zip_clmn_hdr_nm, "Lowest Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

### Create a DataFrame for the Most Expensive Auto Insurance Rates from hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df (Two Cells Above)

In [ ]:
# Splitting Up The Table of The Highest and Lowest Auto Insurance Rate
hghst_avg_annl_auto_insrnc_rates_per_state_df = hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df[["State", "Most Expensive Zip Code", "City.1", "Highest Average Annual Auto Insurance Rates"]]
hghst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Reorganize Column Headers and Rename Some of the Column Headers
hghst_avg_annl_auto_insrnc_rates_per_state_df = hghst_avg_annl_auto_insrnc_rates_per_state_df.rename ({"Most Expensive Zip Code" : zip_clmn_hdr_nm, "City.1" : "City", "Highest Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
hghst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Combine All of the DataFrames into One DataFrame

In [ ]:
# Create a Variable with All of the Average Auto Insurace Tables (DataFrames)
lst_of_tbls = [mst_expnsv_by_state_CarInsurance_df, _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df, _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df, _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df, lwst_avg_annl_auto_insrnc_rates_per_state_df, hghst_avg_annl_auto_insrnc_rates_per_state_df]

# Concatenate All of the Average Auto Insurace Tables Together
avg_auto_insnc_concat_df = pd.concat(lst_of_tbls)
avg_auto_insnc_concat_df


## This Will Allow You to Reset the Index After Concatenating the DataFrames Together
# avg_auto_insnc_concat_df = pd.concat(lst_of_tbls, ignore_index = 'True')

# Data Clean UP

## Delete Dulpicate Data

### Deleted Rows with "Get Your Rates Quote Now" in Them

In [ ]:
# Evaluate Duplicate Data by Displaying the Number of Unique Values in the DataFrame
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

In [ ]:
# Display Rows with "Get Your Rates Quote Now" in Them
avg_auto_insnc_concat_df[(avg_auto_insnc_concat_df[zip_clmn_hdr_nm] == "Get Your Rates Quote Now")]

In [ ]:
# Filtered the DataFrame then Created an Index List of all the Rows with "Get Your Rates Quote Now" in the City Column
drp_gt_your_rts_qute_nw_index = avg_auto_insnc_concat_df[(avg_auto_insnc_concat_df["City"] == "Get Your Rates Quote Now")].index
drp_gt_your_rts_qute_nw_index

In [ ]:
# Used the above Index List to drop all the rows with "Get Your Rates Quote Now"
avg_auto_insnc_concat_df = avg_auto_insnc_concat_df.drop(drp_gt_your_rts_qute_nw_index)
avg_auto_insnc_concat_df

In [ ]:
# Make Sure the Data was Deleted by Displaying the Number of Unique Values in the DataFrameNumber of Unique Values in the DataFrame
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

### Delete the Duplicate Zip Codes from the DataFrame

#### Evaluate Duplicate Data

In [ ]:
# Evaluate Duplicate Data by Displaying the Number of Unique Values in the DataFrame Number
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

#### Delete the Duplicate Rows in The DataFrame

In [ ]:
# Delete Rows with the Same Data
avg_auto_insnc_concat_df_drp_dplcts = avg_auto_insnc_concat_df.drop_duplicates()
avg_auto_insnc_concat_df_drp_dplcts

In [ ]:
# Make Sure the Data was Deleted by Displaying the Number of Unique Values in the DataFrameNumber of Unique Values in the DataFrame
avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts()

#### Reindex the DataFrame

In [ ]:
# Make an List of Values Incrementing by One
avg_auto_insnc_concat_index_list = []

for a in range(len(avg_auto_insnc_concat_df_drp_dplcts)):
#     print(a)
    avg_auto_insnc_concat_index_list.append(a)
avg_auto_insnc_concat_index_list

# Add a New Column Called Index and Append/Populate the Index Column with the Values from the avg_auto_insnc_concat_index_list List.
avg_auto_insnc_concat_df_drp_dplcts["Index"] = avg_auto_insnc_concat_index_list
avg_auto_insnc_concat_df_drp_dplcts

# Make the Index Column the Index for the DataFrame
avg_auto_insnc_concat_df_drp_dplcts = avg_auto_insnc_concat_df_drp_dplcts.set_index("Index")
avg_auto_insnc_concat_df_drp_dplcts

#### Create the .value_counts Series and Convert the Series into a DataFrame

In [ ]:
# Run .value_counts() on the avg_auto_insnc_concat_df DataFrame and Save the .value_count Series 
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs = avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts()
print(type(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs))
print()

# Convert the .value_counts Series into a DataFrame and Save the DataFrame
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df = pd.DataFrame(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs)
print(type(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df))
print()
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df

#### Create a DataFrame of the Zip Codes with a Value Count Greater than One

In [ ]:
# Change the Zip Code Column Header to Value Count
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df.rename ({zip_clmn_hdr_nm : "Value Count"}, axis = 1)
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df

In [ ]:
# Create a New DataFrame that was Filtered the Value Counts Column for Values Greater than 1 
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1 = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df[(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df["Value Count"] > 1)]
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

In [ ]:
# Add a Column Header to the Index Column
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.index.names = [zip_clmn_hdr_nm]
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

# Make a List of Values Incrementing by One
dplcts_val_cnts_index_list = []

for a in range(len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)):
#     print(a)
    dplcts_val_cnts_index_list.append(a)
dplcts_val_cnts_index_list

# Add a New Column Called Index and Append/Populate the Index Column with the Values from the index_list List.
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1["Index"] = dplcts_val_cnts_index_list
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

# Make the Index Column the Index for the DataFrame
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1 = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.reset_index().set_index("Index")
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

#### Delete the Remaining Duplicate Zip Codes

In [ ]:
# Print the length of the Average Auto Insurance DataFrame and Average Auto Insurance Value Counts DataFrame
print(f"This is the length of the Average Auto Insurance DataFrame: {len(avg_auto_insnc_concat_df_drp_dplcts)}")
print()
print(f"This is the length of the Average Auto Insurance Value Counts DataFrame: {len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)}")
print()

# Find And Print the Entire Row for the Remaining Duplicate Zip Codes
for y in range(len(avg_auto_insnc_concat_df_drp_dplcts)):
    
    for h in range(len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)):
        
        if avg_auto_insnc_concat_df_drp_dplcts.iloc[y][zip_clmn_hdr_nm] == avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.iloc[h][zip_clmn_hdr_nm]: 
            print(f"State: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y]['State']}; \tCity: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y]['City']}; \tZip Code: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y][zip_clmn_hdr_nm]}; \tAverage Auto Insurance Rate: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y][avg_auto_insrnc_rate_clmn_hdr_nm]}")

In [ ]:
# Data Types for the Columns in the avg_auto_insnc_concat_df_drp_dplcts DataFrame
avg_auto_insnc_concat_df_drp_dplcts.dtypes

In [ ]:
# Get the Index Number for the Data Filtered on: 48201, Detroit, Blank State, $30,350.09 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_48201_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "48201") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Detroit") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$30,350.09")].index
test = drp_dplct_48201_zip_cd_index[0]
test

In [ ]:
# Get the Index Number for the Data Filtered on: 24450, Lexington, Blank State, $2,018.13 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_24450_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "24450") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Lexington") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$2,018.13")].index
drp_dplct_24450_zip_cd_index[0]

In [ ]:
# Get the Index Number for the Data Filtered on: 06340, Groton, Delaware, $3,896.10 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_06340_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "06340") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Groton") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "Delaware") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$3,896.10")].index
drp_dplct_06340_zip_cd_index[0]

##### Drop the Remaining Duplicate Data

In [ ]:
# Print the Number of Unique Items in avg_auto_insnc_concat_df_drp_dplcts
print("Values Before Dropping the Remain Duplicates")
print(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts())
print()
print()

# Drop the Zip Codes from the Cell Above (Which includes: drp_dplct_48201_zip_cd_index[0], drp_dplct_24450_zip_cd_index[0], drp_dplct_06340_zip_cd_index[0])   
# avg_auto_insnc_concat_df_clnd = avg_auto_insnc_concat_df_drp_dplcts.drop([drp_dplct_48201_zip_cd_index, drp_dplct_24450_zip_cd_index, drp_dplct_06340_zip_cd_index])
avg_auto_insnc_concat_df_clnd = avg_auto_insnc_concat_df_drp_dplcts.drop([drp_dplct_48201_zip_cd_index[0], drp_dplct_24450_zip_cd_index[0], drp_dplct_06340_zip_cd_index[0]])

# Print the Number of Unique Items in avg_auto_insnc_concat_df_clnd, Do this to Make sure the Duplicate Data was Dropped
print("Values After Dropping the Remain Duplicates")
avg_auto_insnc_concat_df_clnd[zip_clmn_hdr_nm].value_counts()

# Add a State to a Blank Row in the State Column

In [ ]:
avg_auto_insnc_concat_df_clnd["City"]

In [ ]:
# Show all the Rows with no Data in the State Column
rows_wth_out_state = avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]
rows_wth_out_state

## Put the State of Michigan into the Rows with Detroit as the City

In [ ]:
# Show all the Rows with Detroit
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["City"] == "Detroit")]

In [ ]:
# Put the State of Michigan into the Rows with Detroit as the City
for x in range(len(avg_auto_insnc_concat_df_clnd)):

    if avg_auto_insnc_concat_df_clnd.iloc[x]["City"] == "Detroit" and avg_auto_insnc_concat_df_clnd.iloc[x]["State"] == "":
        
        avg_auto_insnc_concat_df_clnd.iloc[x]['State'] = "Michigan"
        
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["City"] == "Detroit")]

## Replace the Blank State Row with the State Abbreviations

In [ ]:
# Show all the Rows with no Data in the State Column
rows_wth_out_state_df = avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]
rows_wth_out_state_df

In [ ]:
# Use the Zip Code to Pull the State Abbreviation from the Zip-Codes.com API
for n in range(len(avg_auto_insnc_concat_df_clnd)):
    
    if avg_auto_insnc_concat_df_clnd.iloc[n]["State"] == "":

        zip_code = avg_auto_insnc_concat_df_clnd.iloc[n][zip_clmn_hdr_nm]

#         Make a Request to the Zip-Codes.com API for Each Row without a State
        zip_code_response = requests.get(url + zip_code + key + user_API_key).json()
        print(f"{url}{zip_code}{key}{user_API_key}")
        
#         Input the State Abbreviation into the DataFrame
        avg_auto_insnc_concat_df_clnd.iloc[n]["State"] = zip_code_response["State"]

In [ ]:
# Check to Make Sure All the Rows Have a State
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]

In [ ]:
type(zip_code_response)

In [ ]:
zip_code_response.keys()

In [ ]:
print(json.dumps(zip_code_response, indent =4, sort_keys = True))

## Replace the State Abbrevitions with the Full State Name

### View The State Abbreviations Code Table

In [ ]:
# Table of The State Abbreviations Code Table, Table #0, from app02.clerk.org
State_Abbreviations_Code_Table_df = State_Abbreviations_Code_Table[0]
print(type(State_Abbreviations_Code_Table_df))
print(len(State_Abbreviations_Code_Table_df))
print()
State_Abbreviations_Code_Table_df.head()

In [ ]:
State_Abbreviations_Code_Table_df.keys()

In [ ]:
# Replace all the Abbreviations with the Full State Name 
for t in range(len(State_Abbreviations_Code_Table_df)):

    for i in range(len(avg_auto_insnc_concat_df_clnd)):

        if State_Abbreviations_Code_Table_df.iloc[t]["Code"] == avg_auto_insnc_concat_df_clnd.iloc[i]["State"]:
            avg_auto_insnc_concat_df_clnd.iloc[i]["State"] = State_Abbreviations_Code_Table_df.iloc[t]["Description"]

In [ ]:
# Check to Make Sure All the Abbreviations are gone
avg_auto_insnc_concat_df_clnd["State"].value_counts()

In [ ]:
avg_auto_insnc_concat_df_clnd

In [ ]:
# Check to Make Sure All the Rows Have a State
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]

## Replace the $ and Commas

In [ ]:
# Remove the $ sign and commoas from each row in the Average_Auto_Insurance_Rate column
avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm] = avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm].str.replace("$", "")
avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm] = avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm].str.replace(",", "")
avg_auto_insnc_concat_df_clnd.dtypes

In [ ]:
# Check to make sure the $ and commas were removed
avg_auto_insnc_concat_df_clnd

# Export the Average Auto Insurnace DataFrame to a csv File

In [ ]:
avg_auto_insnc_concat_df_clnd.to_csv(exprt_pth, index=True, header=True)